# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [15]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('/Users/begumyalcin/Desktop/spring 2022/text mining for ai/ba-text-mining-master/lab_sessions/lab4/CONLL2003/', 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {
       # add features
        'words': token,
        'pos': pos
    }
    training_features.append(a_dict)
    training_gold_labels.append(ne_label)

print(training_features[1])
print(training_gold_labels)

{'words': 'rejects', 'pos': 'VBZ'}
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [16]:
### Adapt the path to point to the CONLL2003 folder on your local machine
test = ConllCorpusReader('/Users/begumyalcin/Desktop/spring 2022/text mining for ai/ba-text-mining-master/lab_sessions/lab4/CONLL2003/', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in test.iob_words():
    a_dict = {
       # add features
        'words': token,
        'pos': pos
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

print(test_features[2])
print(test_gold_labels)

{'words': 'JAPAN', 'pos': 'NNP'}
['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [17]:
num_instances_train = len(training_features)
num_instances_test = len(test_features)

In [18]:
num_instances_train

203621

In [19]:
num_instances_test

46435

In [20]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


Counter({1: 2, 2: 2, 3: 1, 5: 1})

In [21]:
frequency_label_train = Counter(training_gold_labels)
frequency_label_train

Counter({'B-ORG': 6321,
         'O': 169578,
         'B-MISC': 3438,
         'B-PER': 6600,
         'I-PER': 4528,
         'B-LOC': 7140,
         'I-ORG': 3704,
         'I-MISC': 1155,
         'I-LOC': 1157})

In [22]:
frequency_label_test = Counter(test_gold_labels)
frequency_label_test

Counter({'O': 38323,
         'B-LOC': 1668,
         'B-PER': 1617,
         'I-PER': 1156,
         'I-LOC': 257,
         'B-MISC': 702,
         'I-MISC': 216,
         'B-ORG': 1661,
         'I-ORG': 835})

**Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?**

There are 203621 instances in training data, and 46435 instances in test data. The data is imbalanced. "O" is the majority in both test and train data. "O" is used much more frequently than others. They have both similar frequency distributions. The number of instances is much more than the test data. 

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [23]:
all_features = training_features + test_features

In [24]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
vec = DictVectorizer()
the_array = vec.fit_transform(all_features).toarray() 

In [12]:
training_onehot = the_array[:len(training_features)]
test_onehot = the_array[len(training_features):]

print('Number of training words =', training_onehot.shape)
print('Number of test words =', test_onehot.shape)

Number of training words = (203621, 27361)
Number of test words = (46435, 27361)


In [13]:
print(training_onehot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
print(test_onehot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [15]:
from sklearn import svm

In [16]:
lin_clf = svm.LinearSVC()

In [17]:
lin_clf.fit(training_onehot, training_gold_labels)

LinearSVC()

In [18]:
pred = lin_clf.predict(test_onehot)
print(pred)

['O' 'O' 'I-PER' ... 'O' 'B-PER' 'O']


In [19]:
from sklearn.metrics import classification_report
report = classification_report(test_gold_labels, pred)
report

'              precision    recall  f1-score   support\n\n       B-LOC       0.81      0.78      0.79      1668\n      B-MISC       0.78      0.66      0.72       702\n       B-ORG       0.79      0.52      0.63      1661\n       B-PER       0.86      0.44      0.58      1617\n       I-LOC       0.62      0.53      0.57       257\n      I-MISC       0.57      0.59      0.58       216\n       I-ORG       0.70      0.47      0.56       835\n       I-PER       0.33      0.87      0.48      1156\n           O       0.98      0.98      0.98     38323\n\n    accuracy                           0.92     46435\n   macro avg       0.72      0.65      0.65     46435\nweighted avg       0.94      0.92      0.92     46435\n'

**Which NERC labels does the classifier perform well on? Why do you think this is the case?
Which NERC labels does the classifier perform poorly on? Why do you think this is the case?**

The classifier performs well on "O" label. Because its presicion recall and f1 score is 0.98. It is because the "O" is the most used label, so the classifier easily finds them. It also performs well on "B-LOC" and "B-PER" labels. 

The classifier performs poorly on "B-ORG" and "B-MISC" labels at it is seen in their f1 scores. It may be because of that they are not commonly used.





**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [1]:
# your code here

import gensim

In [2]:
from nltk.corpus.reader import ConllCorpusReader

# adapt the path to point to the local copy of the nerc_datasets folder
train = ConllCorpusReader('/Users/begumyalcin/Desktop/spring 2022/text mining for ai/ba-text-mining-master/lab_sessions/lab4/CONLL2003/',
                          'train.txt', # this will load the file 'train.txt', for the exercise you also need to load 'test.xt' 
                          ['words', 'pos', 'ignore', 'chunk'])



In [3]:
for token, pos, ne_label in train.iob_words():
    print(token, pos, ne_label) # please represent this information using a dictionary for the feature representation
    break

EU NNP B-ORG


In [4]:
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format("/Users/begumyalcin/Desktop/spring 2022/text mining for ai/ba-text-mining-master/lab_sessions/lab4/CONLL2003/GoogleNews-vectors-negative300.bin", binary=True)  

In [5]:
input_vectors=[]
labels=[]
for token, pos, ne_label in train.iob_words():
    
    if token!='' and token!='DOCSTART':
        if token in word_embedding_model:
            vector=word_embedding_model[token]
        else:
            vector=[0]*300
        input_vectors.append(vector)
        labels.append(ne_label)

In [30]:
test_input_vectors=[]
test_labels=[]
for token, pos, ne_label in test.iob_words():
    
    if token!='' and token!='DOCSTART':
        if token in word_embedding_model:
            vector=word_embedding_model[token]
        else:
            vector=[0]*300
        test_input_vectors.append(vector)
        test_labels.append(ne_label)

In [28]:
print('Last ten labels =', labels[:10])

Last ten labels = ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'B-PER']


In [6]:
from sklearn import svm

lin_clf = svm.LinearSVC()

In [7]:
lin_clf.fit(input_vectors, labels)

LinearSVC()

In [9]:
from collections import Counter 
print(Counter(labels))

Counter({'O': 169578, 'B-LOC': 7140, 'B-PER': 6600, 'B-ORG': 6321, 'I-PER': 4528, 'I-ORG': 3704, 'B-MISC': 3438, 'I-LOC': 1157, 'I-MISC': 1155})


In [31]:
pred = lin_clf.predict(test_input_vectors)
print(pred)

['O' 'O' 'B-LOC' ... 'O' 'B-PER' 'O']


In [32]:
from sklearn.metrics import classification_report
report = classification_report(test_gold_labels, pred)
report

'              precision    recall  f1-score   support\n\n       B-LOC       0.76      0.80      0.78      1668\n      B-MISC       0.72      0.70      0.71       702\n       B-ORG       0.69      0.64      0.66      1661\n       B-PER       0.75      0.67      0.71      1617\n       I-LOC       0.51      0.42      0.46       257\n      I-MISC       0.60      0.54      0.57       216\n       I-ORG       0.48      0.33      0.39       835\n       I-PER       0.59      0.50      0.54      1156\n           O       0.97      0.99      0.98     38323\n\n    accuracy                           0.93     46435\n   macro avg       0.68      0.62      0.64     46435\nweighted avg       0.92      0.93      0.92     46435\n'

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [21]:
import pandas

In [38]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/content/ner_dataset.csv'
kaggle_dataset = pandas.read_csv(path, encoding= 'unicode_escape')

In [39]:
len(kaggle_dataset)

1048575

In [40]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

100000 20000


In [41]:
kaggle_dataset

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [42]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

100000 20000


In [46]:
training_features = []
training_gold_labels = []

for index, instance in df_train.iterrows():    
    a_dict = {
       # add features
        'words': instance['Word'],
        'pos': instance['POS']
    }
    training_features.append(a_dict)
    training_gold_labels.append(instance['Tag'])

In [47]:
test_features = []
test_gold_labels = []

for index, instance in df_test.iterrows():    
    a_dict = {
       # add features
        'words': instance['Word'],
        'pos': instance['POS']
    }
    test_features.append(a_dict)
    test_gold_labels.append(instance['Tag'])

In [48]:
print(training_features[:1])
print(training_gold_labels[:1])
print(test_features)
print(test_gold_labels)

[{'words': 'Thousands', 'pos': 'NNS'}]
['O']
[{'words': '"', 'pos': '``'}, {'words': 'Death', 'pos': 'NN'}, {'words': 'to', 'pos': 'TO'}, {'words': 'America', 'pos': 'NNP'}, {'words': '"', 'pos': '``'}, {'words': 'marched', 'pos': 'VBD'}, {'words': 'through', 'pos': 'IN'}, {'words': 'streets', 'pos': 'NNS'}, {'words': 'Wednesday', 'pos': 'NNP'}, {'words': ',', 'pos': ','}, {'words': 'smashing', 'pos': 'VBG'}, {'words': 'cars', 'pos': 'NNS'}, {'words': ',', 'pos': ','}, {'words': 'damaging', 'pos': 'JJ'}, {'words': 'shops', 'pos': 'NNS'}, {'words': 'and', 'pos': 'CC'}, {'words': 'throwing', 'pos': 'VBG'}, {'words': 'stones', 'pos': 'NNS'}, {'words': 'at', 'pos': 'IN'}, {'words': 'U.S.', 'pos': 'NNP'}, {'words': 'troops', 'pos': 'NNS'}, {'words': '.', 'pos': '.'}, {'words': 'Protests', 'pos': 'NNS'}, {'words': 'erupted', 'pos': 'VBD'}, {'words': 'after', 'pos': 'IN'}, {'words': 'Newsweek', 'pos': 'NNP'}, {'words': 'magazine', 'pos': 'NN'}, {'words': 'reported', 'pos': 'VBD'}, {'words': '

In [49]:
num_instances_train = len(training_features)
num_instances_test = len(test_features)

In [50]:
num_instances_train

100000

In [51]:
num_instances_test

20000

In [52]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


Counter({1: 2, 2: 2, 3: 1, 5: 1})

In [53]:
frequency_label_train = Counter(training_gold_labels)
frequency_label_train

Counter({'O': 84725,
         'B-geo': 3303,
         'B-gpe': 1740,
         'B-per': 1668,
         'I-geo': 690,
         'B-org': 1876,
         'I-org': 1470,
         'B-tim': 1823,
         'B-art': 75,
         'I-art': 43,
         'I-per': 1846,
         'I-gpe': 51,
         'I-tim': 549,
         'B-nat': 30,
         'B-eve': 53,
         'I-eve': 47,
         'I-nat': 11})

In [54]:
frequency_label_test = Counter(test_gold_labels)
frequency_label_test

Counter({'O': 16918,
         'B-geo': 741,
         'I-geo': 156,
         'B-tim': 393,
         'B-org': 397,
         'B-gpe': 296,
         'I-org': 321,
         'B-per': 333,
         'I-per': 319,
         'I-tim': 108,
         'B-nat': 8,
         'I-gpe': 2,
         'I-nat': 4,
         'B-art': 4})

In [55]:
all_features = training_features + test_features

In [56]:
from sklearn.feature_extraction import DictVectorizer

In [57]:
vec = DictVectorizer()
the_array = vec.fit_transform(all_features).toarray() 

In [58]:
training_onehot = the_array[:len(training_features)]
test_onehot = the_array[len(training_features):]

print('Number of training words =', training_onehot.shape)
print('Number of test words =', test_onehot.shape)

Number of training words = (100000, 11897)
Number of test words = (20000, 11897)


In [59]:
print(training_onehot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
print(test_onehot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

In [61]:
lin_clf = svm.LinearSVC()

In [62]:
lin_clf.fit(training_onehot, training_gold_labels)

LinearSVC()

In [63]:
pred = lin_clf.predict(test_onehot)
print(pred)

['O' 'O' 'O' ... 'O' 'O' 'O']


In [64]:
from sklearn.metrics import classification_report
report = classification_report(test_gold_labels, pred)
report

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

'              precision    recall  f1-score   support\n\n       B-art       0.00      0.00      0.00         4\n       B-eve       0.00      0.00      0.00         0\n       B-geo       0.80      0.76      0.78       741\n       B-gpe       0.96      0.92      0.94       296\n       B-nat       1.00      0.50      0.67         8\n       B-org       0.64      0.51      0.57       397\n       B-per       0.81      0.53      0.64       333\n       B-tim       0.91      0.76      0.83       393\n       I-art       0.00      0.00      0.00         0\n       I-eve       0.00      0.00      0.00         0\n       I-geo       0.74      0.50      0.60       156\n       I-gpe       1.00      0.50      0.67         2\n       I-nat       0.80      1.00      0.89         4\n       I-org       0.65      0.44      0.53       321\n       I-per       0.42      0.90      0.57       319\n       I-tim       0.41      0.08      0.14       108\n           O       0.98      0.99      0.99     16918\n\n    a

B-gpe and B-tim labels have the highest F1-score. Model performs well on the B-gpe and B-tim labels. B-geo, B-org, and B-per labels have average f1 score, model acts moderately well on these labels. B-art, B-eve, I-eve, I-art has f1 score of 0, model performs bad on these labels. I-per and I-nat labels are also can be detected easily compared to other labels.

## End of this notebook